In [1]:
from fastai.imports import *

from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *


In [2]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import pickle

with open('iiitd/train_image.pkl', 'rb') as f:
    data = pickle.load(f)
with open('iiitd/train_label.pkl', 'rb') as f:
    label = pickle.load(f)

In [4]:
im = Image.open('iiitd/Prd.AcousticRsrchGold_01_large.png')
a = np.asarray(im)

In [5]:
df=pd.DataFrame({'id':[],'label':[]})
df

,id,label


In [6]:
ids=[]
labels=[]
for i in range(8000):
    image=np.reshape(data[i],(28,28))
    image=np.asarray(image)
    image=image.astype(a.dtype)
    image=Image.fromarray(image)
    x="iiitd/train/"+str(i)+".png"
    ids.append(i)
    labels.append(label[i])
    image.save(x)

In [7]:
for i in range(2000):
    image=np.reshape(data[i],(28,28))
    image=np.asarray(image)
    image=image.astype(a.dtype)
    image=Image.fromarray(image)
    x="iiitd/test/"+str(i)+".png"
    image.save(x)

In [8]:
df['id']=ids
df['label']=labels
df.head()

,id,label
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [9]:
df.pivot_table(index="label", aggfunc=len).sort_values('id', ascending=False)

,id
label,
0,2000
2,2000
3,2000
6,2000


In [10]:
df.to_csv('iiitd/labels.csv',index=False)

In [11]:
# image=np.reshape(data[1],(28,28))
# print(label[7100])
# plt.imshow(image)

In [12]:
PATH = "iiitd/"
sz = 224
arch = resnet34
bs = 32

In [13]:
n=8000
val_idxs = get_cv_idxs(n)

In [14]:
tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
data = ImageClassifierData.from_csv(PATH, 'train', f'{PATH}labels.csv', test_name='test', # we need to specify where the test set is if you want to submit to Kaggle competitions
                                   val_idxs=val_idxs, suffix='.png', tfms=tfms, bs=bs)

In [15]:
fn = PATH + data.trn_ds.fnames[0]; fn

'iiitd/train/0.png'

In [16]:
img = PIL.Image.open(fn); img

In [17]:
img.size

(28, 28)

In [18]:
learn = ConvLearner.pretrained(arch, data, precompute=True)

In [19]:
learn.summary()

OrderedDict([('Conv2d-1',
              OrderedDict([('input_shape', [-1, 3, 224, 224]),
                           ('output_shape', [-1, 64, 112, 112]),
                           ('trainable', False),
                           ('nb_params', 9408)])),
             ('BatchNorm2d-2',
              OrderedDict([('input_shape', [-1, 64, 112, 112]),
                           ('output_shape', [-1, 64, 112, 112]),
                           ('trainable', False),
                           ('nb_params', 128)])),
             ('ReLU-3',
              OrderedDict([('input_shape', [-1, 64, 112, 112]),
                           ('output_shape', [-1, 64, 112, 112]),
                           ('nb_params', 0)])),
             ('MaxPool2d-4',
              OrderedDict([('input_shape', [-1, 64, 112, 112]),
                           ('output_shape', [-1, 64, 56, 56]),
                           ('nb_params', 0)])),
             ('Conv2d-5',
              OrderedDict([('input_shape', [-1, 64, 56, 

In [20]:
learn.fit(1e-2, 5)

epoch      trn_loss   val_loss   accuracy                     
    0      0.622193   0.492377   0.82125   
    1      0.543661   0.474014   0.838125                     
    2      0.537946   0.430533   0.841875                     
    3      0.501013   0.441953   0.835625                     
    4      0.489993   0.424403   0.84375                      



[array([0.4244]), 0.84375]

In [21]:
learn.fit(1e-2, 3, cycle_len=1, cycle_mult=2)

epoch      trn_loss   val_loss   accuracy                     
    0      0.400465   0.406516   0.849375  
    1      0.455495   0.39979    0.853125                     
    2      0.398791   0.392046   0.849375                     
    3      0.439729   0.403132   0.845625                     
    4      0.4137     0.402465   0.850625                     
    5      0.389239   0.390326   0.85                         
    6      0.377234   0.390888   0.8525                       



[array([0.39089]), 0.8525]

In [22]:
learn.fit(1e-2, 3, cycle_len=1, cycle_mult=2)

epoch      trn_loss   val_loss   accuracy                     
    0      0.359637   0.409245   0.85      
    1      0.347397   0.397277   0.85                         
    2      0.365173   0.380608   0.8575                       
    3      0.384618   0.394158   0.8525                       
    4      0.358484   0.394584   0.85125                      
    5      0.362159   0.391314   0.855                        
    6      0.313024   0.382509   0.8525                       



[array([0.38251]), 0.8525]

In [24]:
learn.fit(1e-2, 3, cycle_len=2,cycle_mult=3)

epoch      trn_loss   val_loss   accuracy                     
    0      0.332806   0.384319   0.860625  
    1      0.273926   0.383969   0.86                         
    2      0.30565    0.402177   0.856875                     
    3      0.319247   0.406532   0.85375                      
    4      0.296404   0.380897   0.85875                      
    5      0.304741   0.40021    0.853125                     
    6      0.270838   0.38282    0.859375                     
    7      0.251687   0.392855   0.853125                     
    8      0.295372   0.412746   0.8475                       
    9      0.304616   0.415646   0.851875                     
    10     0.298587   0.390982   0.861875                     
    11     0.29016    0.405775   0.855                        
    12     0.282446   0.40518    0.86375                      
    13     0.277239   0.409191   0.8525                       
    14     0.247931   0.395898   0.855625                     
    15     

[array([0.40379]), 0.858125]

In [27]:
learn.precompute = False

In [26]:
learn.fit(1e-2, 3)

epoch      trn_loss   val_loss   accuracy                    
    0      0.603857   0.453469   0.840625  
    1      0.531741   0.448709   0.8325                      
    2      0.511144   0.435051   0.840625                    



[array([0.43505]), 0.840625]